<img src="https://www.108themes.com/hubble/slider/11.jpg" style="border-radius:50%">
<h1 style="font-size:50px;color:#35858B;text-align:center"><strong>Titanic</strong> <strong style="color:black">Spaceship Story</strong></h1>

<i style="font-size:100%">&nbsp;&nbsp;&nbsp;&nbsp;In 2912, there was a spaceship of 13,000 passengers launched to tranport emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars...And this was the first voyage from its kind.</i>
<i style="font-size:100%">On its way and nothing happen, suddenly it has been collided with a spacetime anomaly hidden within a dust cloud, which ends up with what happen to it since 1000 years ago, almost half of the passengers were transported to an alternate dimension!</i>


<a href="https://www.kaggle.com/mostafaalaa123/how-terrifying-it-is-to-be-lost-in-space-ml-02?scriptVersionId=89169283">Second Part (ML)</a>

### This notebook is the first part (EDA), the second part will come soon

In [ ]:
from IPython.core.display import display, HTML, Javascript

# ----- Notebook Theme -----
color_map = ['#35858B', '#f6f0e8', '#ecdfd0', '#d9c1a2', '#AEFEFF', '#072227', 
                        '#35858B', '#8d6213', '#117a65', '#7a5211', '#4FBDBA']

prompt = color_map[-1]
main_color = color_map[0]
strong_main_color = color_map[1]
custom_colors = [strong_main_color, main_color]

css_file = ''' 

div #notebook {
background-color: white;
line-height: 20px;
}

#notebook-container {
%s
margin-top: 2em;
padding-top: 2em;
border-top: 4px solid %s; /* light orange */
-webkit-box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5); /* pink */
    box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5); /* pink */
}

div .input {
margin-bottom: 1em;
}

.rendered_html h1, .rendered_html h2, .rendered_html h3, .rendered_html h4, .rendered_html h5, .rendered_html h6 {
color: %s; /* light orange */
font-weight: 600;
}

div.input_area {
border: none;
    background-color: %s; /* rgba(229, 143, 101, 0.1); light orange [exactly #E58F65] */
    border-top: 2px solid %s; /* light orange */
}

div.input_prompt {
color: %s; /* light blue */
}

div.output_prompt {
color: %s; /* strong orange */
}

div.cell.selected:before, div.cell.selected.jupyter-soft-selected:before {
background: %s; /* light orange */
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border-color: %s; /* light orange */
}

.edit_mode div.cell.selected:before {
background: %s; /* light orange */
}

.edit_mode div.cell.selected {
border-color: %s; /* light orange */

}
'''
def to_rgb(h): 
    return tuple(int(h[i:i+2], 16) for i in [0, 2, 4])

main_color_rgba = 'rgba(%s, %s, %s, 0.1)' % (to_rgb(main_color[1:]))
open('notebook.css', 'w').write(css_file % ('width: 95%;', main_color, main_color, main_color_rgba, main_color,  main_color, prompt, main_color, main_color, main_color, main_color))

def nb(): 
    return HTML("<style>" + open("notebook.css", "r").read() + "</style>")
nb()

<h1 style="font-size:30px;color:#35858B;text-align:left"><strong>Factors</strong> <strong style="color:black">Effeciencies</strong></h1> 

&nbsp;&nbsp;&nbsp;At this phase, what we're going do, God willing, is to go through each factor in the dataset and see its distribution and how it affects the target as we can also extract from it another columns or even leave it without putting it into the model.

In [ ]:
# Initial Cell

# Importing Libraries

# For Data Preprocessing and Analysis
import pandas as pd
import numpy as np

# For Data Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Removing Warnings
import warnings
warnings.filterwarnings('ignore')


# Data
train_df = pd.read_csv('../input/spaceship-titanic/train.csv')
test_df = pd.read_csv('../input/spaceship-titanic/test.csv')
sample_sub = pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

> # PassengerId

**Brief Definition:** First, as PassengerId written in the form gggg_pp, such as gggg number of the group he is travelling with and pp is his/her id in this group, we're going to separate these two numbers into two columns, #Group, and id.

#### Let's see the relation between "#Group" and trasnportation

In [ ]:
train_df['#Group'], train_df['id'] = train_df['PassengerId'].str[:4].astype('int'), train_df['PassengerId'].str[5:].astype('int')
test_df['#Group'], test_df['id'] = test_df['PassengerId'].str[:4].astype('int'), test_df['PassengerId'].str[5:].astype('int')
def plot_numerical(df, var, title=None):
    
    # data for plotting (happiness wold countries)
    trasported = train_df[train_df['Transported'] == True][var]
    non_trasported = train_df[train_df['Transported'] == False][var]

    fig = go.Figure()
    fig.add_trace(go.Violin(x=trasported, line_color='red', name='Tranported', y0=0))
    fig.add_trace(go.Violin(x=non_trasported, line_color='lightseagreen', name= 'Non-Transported', y0=0))

    fig.update_traces(orientation='h', side='positive', meanline_visible=False)
    fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False)

    fig.update_layout(title=f'<b>How {var} affects Transporting<b>' if not None else title,
                      xaxis_title=var,
                      titlefont={'size': 24},
                      template="plotly_dark",
                      showlegend=True,
                      font=dict(
                          color ='white',
                          )
                      )
    
    fig.show()
    
plot_numerical(train_df, '#Group')

### So as we can see we can divide this graph into three ranges:  

1. where the number of transported passengers are less due to #group which is low  <b>(less #group, higher non-transported)</b>
2. #group became higher then the non-transported passengers are lesser <b>(medium #group, higher transported)</b>
3. #group very high #group, higher number of non-transported passengers <b>(high #group, higher non-transported)</b>

<b><mark>Result 1</mark> Three columns will be added, one for each category

In [ ]:
# data to plot:  age distribution survived vs not-sutvived
trasported = train_df[train_df['Transported'] == True]['#Group']
non_trasported = train_df[train_df['Transported'] == False]['#Group']


group_labels = ['Transported', 'Non-Transported']
fig = ff.create_distplot([trasported, non_trasported],
                         group_labels, 
                         show_hist=False, 
                         show_rug=False,
                         )
fig.add_vrect(
    x0=2.01, x1=3400,
    annotation_text="Low #Group", annotation_position="top",
    fillcolor="skyblue", opacity=0.5,
    layer="below", line_width=0,
),
fig.add_vrect(
    x0=3401, x1=7350,
    annotation_text="Medium #Group", annotation_position="top",
    fillcolor="Salmon", opacity=0.5,
    layer="below", line_width=0,
),
fig.add_vrect(
    x0=7351, x1=9300,
    annotation_text="High #Group", annotation_position="top",
    fillcolor="lightgray", opacity=0.5,
    layer="above", line_width=0,
),
fig.update_layout(title='<b>#Group Ranges due to safety from transporting<b>',
                  xaxis_title='<b>#Group', 
                  yaxis_title='<b>Probability Density [-]',
                  titlefont={'size': 24},
                  font_family = 'San Serif',
                  template="simple_white",
                  showlegend=True,
                  paper_bgcolor="white",
                  font=dict(
                      color ='black',
                      ),
                  legend=dict(
                      orientation="v",
                      y=1, 
                      yanchor="bottom", 
                      x=1.0, 
                      xanchor="right",)   
 )

fig.show()

#### Safe, Danger, a bit safe
Then let's see the relation between id and transportation

In [ ]:
def plot_categorical(df, var):
    fig = px.histogram(df, x=var, y=None, color="Transported",
                    histnorm='percent',
                    color_discrete_map={ 
                        False: "skyblue", True: "#e28679"
                    },
                    template="simple_white"
                    )
    fig.update_layout(title=f"<b>Relation Between Transporting and {var.title()}</b>", 
                      font_family="San Serif",
                      bargap=0.2,
                      barmode='group',
                      titlefont={'size': 24},
                      legend=dict(
                      orientation="v", 
                          y=1, 
                          yanchor="top", 
                          x=1.250, 
                          xanchor="right",)                 
                      )
    fig.show()
    
plot_categorical(train_df, 'id')

#### Also, id must be divided into ranges, but two ranges:
1. when Id equals 1
2. when Id equals 2 or 3

<b><mark>Result 2</mark> We should make new two columns, one for each range

> # Home Planet

<b>Brief Definition</b>: The planet the passenger departed from, typically their planet of permanent residence.

In [ ]:
plot_categorical(train_df, 'HomePlanet')

#### So we can see that the most safe planet is Earth and the most dangerous one is Europa, Mars is nutral.

<b><mark>Result 3</mark>: So they can be divided into two columns one for Europa and another for Earth

> # CryoSleep

<b>Brief Definition</b>: Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

In [ ]:
plot_categorical(train_df, 'CryoSleep')

#### Passengers elected to be put into suspended animation are highly affected to be transported with a huge ratio.

<b><mark>Result 4</mark>

> # Cabin

<b>Brief Definition</b>: The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.

At this Notebook, three columns will be added, deck, num, and side

In [ ]:
# Separation
train_df['deck'], train_df['num'], train_df['side'] = train_df['Cabin'].str[0], train_df['Cabin'].str[2:-2], train_df['Cabin'].str[-1]
test_df['deck'], test_df['num'], test_df['side'] = test_df['Cabin'].str[0], test_df['Cabin'].str[2:-2], test_df['Cabin'].str[-1]

In [ ]:
plot_categorical(train_df, 'deck')

### This can be divided into three categories:
1. **Safe Decks:** F, E, and D
2. **Dangerous Decks:** B, C
3. **Nutral Decks:** The rest

<b><mark>Result 5</mark> we should make new two columns one for each category Safe_Deck and Danger_Deck

In [ ]:
plot_categorical(train_df, 'side')

#### Port S is more dangerous than port P, so this column will not be thrown
<b><mark>Result 6</mark>

> # Age

In [ ]:
# train_df.loc[train_df.loc[:,"Age"].isnull(),"Age"]=train_df.loc[:,"Age"].median()
# test_df.loc[test_df.loc[:,"Age"].isnull(),"Age"]=test_df.loc[:,"Age"].median()

In [ ]:
plot_numerical(train_df, 'Age')

In [ ]:
plot_categorical(train_df, 'Age')

#### So new columns will be added which is older_than_9, and is_between_19_and_24
##### As people have less than or equal 9 have higher ratio of being transported and the opposite with the second.

<b><mark>Result 7</mark>

> # VIP

<b>Brief Definition: </b>Whether the passenger has paid for special VIP service during the voyage.

In [ ]:
plot_categorical(train_df, 'VIP')

#### As shown above VIP is not an effective factor, so it can be thrown
<b><mark>Result 8</mark>

> # RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck

<b>Brief Definition:</b> Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.

In [ ]:
train_df['Total Payments'] = train_df['RoomService'] + train_df['FoodCourt'] + train_df['ShoppingMall'] + train_df['Spa'] + train_df['VRDeck']
plot_numerical(train_df[train_df['Total Payments'] < 5000], 'Total Payments')

#### It's clear that when your total payments are high your chance to being safe (Non-Transported) is high
<b><mark>Result 9</mark>

> # Destination

In [ ]:
plot_categorical(train_df, 'Destination')

#### It's clear that Adding the last value will not be a good decision so we will make a column for each of the first
<b><mark>Result 10</mark>

<h1 style="font-size:30px;color:#35858B;text-align:left"><strong style="color:black">Don't Forget to </strong><strong>Upvote</strong></h1> 